## INSTALLING PACKAGES AND LIBRARIES

In [1]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install -U sentence-transformers
!{sys.executable} -m pip install seaborn
!{sys.executable} -m pip install tqdm

Requirement already up-to-date: sentence-transformers in c:\users\augus\appdata\local\programs\python\python37\lib\site-packages (2.2.2)


You should consider upgrading via the 'c:\Users\augus\AppData\Local\Programs\Python\Python37\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\Users\augus\AppData\Local\Programs\Python\Python37\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\Users\augus\AppData\Local\Programs\Python\Python37\python.exe -m pip install --upgrade pip' command.


In [2]:
import relief
import utils
import pandas as pd
import numpy as np

# Visualization
from tqdm import tqdm
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
# Set Plot Theme
sns.set_palette([
    "#30a2da",
    "#fc4f30",
    "#e5ae38",
    "#6d904f",
    "#8b8b8b",
])
# Warnings
import warnings
warnings.filterwarnings('ignore')

c:\Users\augus\AppData\Local\Programs\Python\Python37\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## WOMEN E COMMERCE

In [ ]:
df = pd.read_csv(r"C:\Users\augus\Desktop\projectTesis\datasets\women_e_commerce\Womens Clothing E-Commerce Reviews.csv")
df.head()

Preproccesing steps:

In [ ]:
# contain redundant information about the index of the dataset
df = df.drop(columns = "Unnamed: 0")
# reformat the column names
df.columns = (df.columns
                .str.replace(' ', '_')
                .str.lower()
             )
print(df.columns)
df.shape

In [ ]:
df.info()

In [ ]:
total_rows = len(df)
missing_rows = df['review_text'].isna().sum()
missing_rows_title = df['title'].isna().sum()

percentage_missing = (missing_rows / total_rows) * 100
percentage_missing_title = (missing_rows_title/total_rows)*100
print(f'Percentage of Missing Values for review text column: {percentage_missing:.2f}%')
print(f'Percentage of Missing Values for title column: {percentage_missing_title:.2f}%')

# Drop rows with missing values
df = df.dropna(subset=['review_text'])
#drop column review title.
df = df.drop(columns = "title")

Since the percentage of missing values for the Review text column is 3.60% we drop those samples from the dataset.    
As a rule of thumb: Drop rows if they represent less than 10% of the original dataset. 

In the case for the title column, it has 16.22% of missing values, and it contains only small text summarizing the actual review for that reason we drop the column.


In [ ]:
df.isna().sum()

In [ ]:
df['review_text'] = df['review_text'].astype(str)
df.shape

In [ ]:
unique_division_name = df['division_name'].unique()
unique_department_name = df['department_name'].unique()
unique_class_name = df['class_name'].unique()

# Replacing "Initmitates" with "Intimates" in 'text_column'
df['division_name'] = df['division_name'].str.replace('Initmates', 'Intimates')
df['department_name'] = df['department_name'].str.replace('Intimate', 'Intimates')

# Convert 'category_column' to the 'category' data type
df['division_name'] = df['division_name'].astype('category')
df['department_name'] = df['department_name'].astype('category')
df['class_name'] = df['class_name'].astype('category')

print("Unique values in the unique_division_name:", unique_division_name)
print("Unique values in the unique_department_name:", unique_department_name)
print("Unique values in the unique_class_name:", unique_class_name)

In [ ]:
# the label is recommmended_ind, the rest are the features.
y = df.pop("recommended_ind")

The clothing_id is a numberical identifier for a particular clothing item, a named category hence we are going to their it as a categorical value.  
The rating: integer values from 1 to 5, since their values holds an implied order, we are going to treat as numerical.  
recommended_ind: is the target label.

In [ ]:
# only 3 types: numerical, nominal and text in the same order as the dataframe
relief_column_types = ["nominal", "numerical", "text", "numerical", "numerical", "nominal", "nominal","nominal"]

In [ ]:
#eventhough it is a binary classification problem we hand to hand the missing values and be more robust using K=10 instead of 1.
#relief( m: int, algorithm: str, text_distance_funct: str = "sbert", k=10)
relief_women = relief.Relief(2,"ReliefF")

In [ ]:
df = df.head(1000)
relief_women.fit(df, y, relief_column_types)

In [ ]:
relief_women.history

In [ ]:
plt.bar(df.columns, relief_women.weights)
# Add labels and title
plt.xlabel('attributes')
plt.ylabel('quality estimation')
plt.title('ReliefF estimations')

## 2nd DATASET - PETFINDER MULTICLASSIFICATION

In [3]:
# set the max columns to none
pd.set_option('display.max_columns', None)
df = pd.read_csv(r"C:\Users\augus\Desktop\projectTesis\datasets\train_pet_finder.csv")
df.head()

,Unnamed: 0,Unnamed: 0.1,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,FurLength,Vaccinated,Dewormed,Sterilized,Health,Quantity,Fee,State,RescuerID,VideoAmt,Description,PetID,PhotoAmt,AdoptionSpeed
0,10318,10318,2,Xiao Li,2,Domestic Short Hair,NaN,2,Brown,White,NaN,2,1,2,1,2,1,1,0,Selangor,744fa4278196568a18fcf5cd7d324ed2,0,Xiao Li was rescued when the local council rai...,d5c68b3bf,4.0,3
1,9789,9789,1,Puppy Girl,3,Mixed Breed,Rottweiler,2,Black,Brown,NaN,2,1,1,1,2,1,1,0,Kuala Lumpur,b770bac0ca797cf1433c48a35d30c4cb,0,Active and healthy. Already vaccine and deworm...,b968b85c1,2.0,3
2,11994,11994,2,Cookie And Cream,2,Domestic Short Hair,NaN,3,Yellow,Cream,White,1,1,2,1,2,1,2,0,Selangor,3b36a2f0ba551f7d0276cd1741369c37,0,These kittens were found behind the alley of m...,0cf645af3,3.0,1
3,5451,5451,1,Suki,36,Maltese,Poodle,2,White,NaN,NaN,1,2,1,1,1,1,1,100,Selangor,a63ec365c8c0d28ff01d2cb1b31406cb,0,Friendly..loving..love to play..potty trained,804e56f9b,4.0,1
4,9845,9845,1,A3,1,Mixed Breed,Mixed Breed,1,Black,NaN,NaN,2,2,2,1,2,1,1,0,Selangor,eba983b85e34963999d476ccada91619,0,For Adoption,b414c5dd9,3.0,1


In [4]:
print(df.columns)
print(df.shape)

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Type', 'Name', 'Age', 'Breed1', 'Breed2',
       'Gender', 'Color1', 'Color2', 'Color3', 'MaturitySize', 'FurLength',
       'Vaccinated', 'Dewormed', 'Sterilized', 'Health', 'Quantity', 'Fee',
       'State', 'RescuerID', 'VideoAmt', 'Description', 'PetID', 'PhotoAmt',
       'AdoptionSpeed'],
      dtype='object')
(11993, 26)


In [5]:
# Extracting Missing Count and Unique Count by Column
unique_count = []
for x in df.columns:
    unique_count.append([x,len(df[x].unique()),df[x].isnull().sum()])
pd.DataFrame(unique_count, columns=["Column","Unique","Missing"]).set_index("Column").T

Column,Unnamed: 0,Unnamed: 0.1,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,FurLength,Vaccinated,Dewormed,Sterilized,Health,Quantity,Fee,State,RescuerID,VideoAmt,Description,PetID,PhotoAmt,AdoptionSpeed
Unique,11993,11993,2,7488,104,165,127,3,7,7,6,4,3,3,3,3,3,19,65,14,4781,9,11254,11993,31,5
Missing,0,0,0,1000,0,4,8625,0,0,3538,8442,0,0,0,0,0,0,0,0,0,0,0,7,0,0,0


In [6]:
# memory usage of each column in Bytes.
print(f"total amount of memory usage in Bytes: {df.memory_usage().sum()}")
df.memory_usage()

total amount of memory usage in Bytes: 2494672


Index              128
Unnamed: 0       95944
Unnamed: 0.1     95944
Type             95944
Name             95944
Age              95944
Breed1           95944
Breed2           95944
Gender           95944
Color1           95944
Color2           95944
Color3           95944
MaturitySize     95944
FurLength        95944
Vaccinated       95944
Dewormed         95944
Sterilized       95944
Health           95944
Quantity         95944
Fee              95944
State            95944
RescuerID        95944
VideoAmt         95944
Description      95944
PetID            95944
PhotoAmt         95944
AdoptionSpeed    95944
dtype: int64

In [7]:
df = df.drop(columns = ['Unnamed: 0', 'Unnamed: 0.1', 'RescuerID', 'PetID', 'Color2', 'Color3', 'Breed2','Name', 'VideoAmt'])
# Drop rows with missing values
df = df.dropna(subset=['Description'])
df = df.dropna(subset=['Breed1'])
unique_count = []
for x in df.columns:
    unique_count.append([x,len(df[x].unique()),df[x].isnull().sum()])
pd.DataFrame(unique_count, columns=["Column","Unique","Missing"]).set_index("Column").T

Column,Type,Age,Breed1,Gender,Color1,MaturitySize,FurLength,Vaccinated,Dewormed,Sterilized,Health,Quantity,Fee,State,VideoAmt,Description,PhotoAmt,AdoptionSpeed
Unique,2,104,164,3,7,4,3,3,3,3,3,19,65,14,9,11249,31,5
Missing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [8]:
# Maturity size, fur length, healhty - categorical but treat it as numerical, since their is an ordinal behavior
# treat Breed1 and description as text data type
df['Type'] = df['Type'].astype(float)
df['Gender'] = df['Gender'].astype(float) 
df['Color1'] = df['Color1'].astype(str) 
df['Vaccinated'] = df['Vaccinated'].astype(float) 
df['Dewormed'] = df['Dewormed'].astype(float) 
df['Sterilized'] = df['Sterilized'].astype(float)
 
df['State'] = df['State'].astype(str)
df['Breed1'] = df['Breed1'].astype(str)
df['Description'] = df['Description'].astype(str)

# the label is recommmended_ind, the rest are the features.
y = df.pop("AdoptionSpeed")
df.shape

(11982, 17)

In [10]:
# only 3 types: numerical, nominal and text in the same order as the dataframe
relief_column_types = ["nominal", "numerical", "text","nominal","nominal","numerical","numerical","nominal","nominal","nominal","numerical", "numerical", "numerical","nominal","numerical","text","numerical"]

#relief( m: int, algorithm: str, text_distance_funct: str = "sbert", k=10)
relief_pet = relief.Relief(5,"ReliefF")
df = df.head(100)
relief_pet.fit(df, y, relief_column_types)
relief_pet.history

The text embeddings for column Breed1 were created.
The text embeddings for column Description were created.
Starting ReliefF...


5it [00:06,  1.36s/it]

ReliefF Completed...


array([[-0.0397585 ,  0.01582021, -0.00657498, -0.01268006, -0.00485146,
        -0.02635178, -0.01138491, -0.05036119, -0.01121393,  0.03978628,
         0.00689891, -0.03119128, -0.02239989, -0.00167557,         nan,
         0.00915175,  0.00324578],
       [-0.02735308,  0.0264376 ,  0.00838129,  0.00432098, -0.03783075,
         0.00056942,  0.00940185, -0.01532474,  0.02382251,  0.08089469,
         0.01436854, -0.01505522, -0.00698263, -0.05710309,         nan,
         0.01350341,  0.00112545],
       [ 0.00686577,  0.0422067 ,  0.00673804, -0.00835908, -0.01625777,
        -0.02578236, -0.00198306, -0.00840232,  0.01260858,  0.12068097,
         0.02126745, -0.0462465 , -0.02938252, -0.05707958,         nan,
         0.01729954,  0.00296815],
       [ 0.04108462,  0.04482051,  0.0050948 , -0.01826504, -0.02110923,
        -0.05213415, -0.01336797, -0.05876351,  0.00139464,  0.05745592,
         0.02816636, -0.07743778, -0.0517824 , -0.05705607,         nan,
         0.01624726

In [14]:
relief_pet.history[0][14]

nan